In [1]:
import os
import requests
from bs4 import BeautifulSoup
from docx import Document

def clean_filename(title):
    invalid_chars = '\\/:*?"<>|'
    cleaned_title = ''.join(c for c in title if c not in invalid_chars)
    return cleaned_title.strip()

url = "https://blogs.exeter.ac.uk/readingforlife/2017/02/12/a-blog-post-about-happiness-by-emma-bingham/"
req = requests.get(url)
soup = BeautifulSoup(req.content, "html.parser")

title = soup.title.text

doc = Document()
doc.add_heading(title, level=1)

paragraphs = soup.find_all('p')
for paragraph in paragraphs:
    doc.add_paragraph(paragraph.text)

output_folder = "E:\minor_project\Data"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

cleaned_title = clean_filename(title)
output_doc_file = os.path.join(output_folder, f"{cleaned_title}.docx")
doc.save(output_doc_file)

print(f"Data saved to {output_doc_file}")



Data saved to E:\minor_project\Data\A blog post about happiness, by Emma Bingham - Reading for Life.docx


In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from docx import Document

def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

def predict_mood(document_path):
    doc = Document(document_path)
    content = ''
    for paragraph in doc.paragraphs:
        content += paragraph.text + ' '
    preprocessed_content = preprocess_text(content)
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(preprocessed_content)
    if sentiment_scores['compound'] >= 0.3:
        return 'Happy'
    elif sentiment_scores['compound'] <= -0.3:
        return 'Sad'
    else:
       
        if 'love' in preprocessed_content.lower():
            return 'Love'
        else:
            return 'Neutral'
          
document_path = output_doc_file
predicted_mood = predict_mood(document_path)
print("Predicted Mood:", predicted_mood)


Predicted Mood: Happy


In [3]:
import pandas as pd


Music_Player = pd.read_csv("E:\minor_project\Data\data_moods.csv")
Music_Player = Music_Player[['name', 'artist', 'mood', 'popularity']]

def Recommend_Songs(pred_class):
    if pred_class == 'Happy':
        Play = Music_Player[Music_Player['mood'] == 'Happy']
        Play = Play.sort_values(by="popularity", ascending=False)
        Play = Play[:5].reset_index(drop=True)
        display(Play)
    elif pred_class == 'Sad':
        Play = Music_Player[Music_Player['mood'] == 'Sad']
        Play = Play.sort_values(by="popularity", ascending=False)
        Play = Play[:5].reset_index(drop=True)
        display(Play)


current_mood = predicted_mood 
Recommend_Songs(current_mood)


,name,artist,mood,popularity
0,Pumped Up Kicks,Foster The People,Happy,84
1,Africa,TOTO,Happy,84
2,Take on Me,a-ha,Happy,84
3,Highway to Hell,AC/DC,Happy,83
4,Here Comes The Sun - Remastered 2009,The Beatles,Happy,83
